In [1]:
%load_ext autoreload
%autoreload 2

import requests 
import json 
import os 
import glob

import numpy as np 
import pandas as pd 
import time

import sys
sys.path.insert(1, '../python')

import twitter_api

from tqdm import tqdm
from datetime import datetime
import duckdb

In [2]:
api_key, api_key_secret, bearer_token = twitter_api.get_auth_keys("../../twitter_api_keys.json")
params = twitter_api.create_param_dict()
headers = twitter_api.create_header_dict(bearer_token)

In [58]:
columns = ["tweet_id", "user_id", "bullying_trace", "author_role", "teasing"]
df = pd.read_csv("../../data/bullyingV2.0/tweetID_userID_category_authorRole_tease.csv", names=columns)
tweet_ids = df.tweet_id

twitter_api.get_tweets_from_id(tweet_ids, bearer_token, params, headers, save_folder = "../../data/")

NameError: name 'bearer_token' is not defined

In [40]:
columns = ["tweet_id", "label"]
df = pd.read_csv("../../data/NAACL_SRW_2016.csv", names=columns)

tweet_ids = df.tweet_id
twitter_api.get_tweets_from_id(tweet_ids, bearer_token, params, headers, save_folder = "../../data/Waseem/")

100%|██████████| 170/170 [03:54<00:00,  1.38s/it]


In [61]:
link = "https://bitbucket.org/ssalawu/cyberbullying-twitter/raw/f117853efbcc5e5d1f23a2d65f59c2855e264db3/dataset.csv"
df = pd.read_csv(link)
df.Id = df.Id.str.replace("'", "").astype('str')
df.to_csv("../../data/Salawu.csv", index=False)

In [17]:
tweet_ids = df.Id.astype("int64").astype("str")
twitter_api.get_tweets_from_id(tweet_ids, bearer_token, params, headers, save_folder = "../../data/Salawu/")

 43%|████▎     | 271/626 [02:39<02:40,  2.21it/s] 

Timed out, sleeping until 2022-06-25 00:17:58


 91%|█████████ | 570/626 [11:59<00:25,  2.20it/s]    

Timed out, sleeping until 2022-06-25 00:33:08


100%|██████████| 626/626 [25:19<00:00,  2.43s/it]   


In [20]:
json_files = glob.glob("../../data/Salawu/*.json")
successes = 0
errors = 0 

for fp in json_files: 
    with open(fp) as data: 
        data = json.load(data)
        try: 
            successes += len(data["data"])
            errors += len(data["errors"])
        except KeyError: 
            continue

print("Successes: ", successes)
print("Errors: ", errors)

Successes:  8553
Errors:  54597


In [45]:
naacl_srw_2016 = pd.read_csv("../../data/NAACL_SRW_2016.csv", names = ["tweet_id", "label"])

json_files = glob.glob("../../data/Waseem/*.json")
successes = 0
errors = 0 

tweet_frames = []
user_frames = []
geo_frames = []

for fp in json_files: 
    with open(fp) as data: 
        data = json.load(data)
        try: 
            tweet_frames.append(pd.DataFrame(data["data"]))
            user_frames.append(pd.DataFrame(data["includes"]["users"]))
            geo_frames.append(pd.DataFrame(data["includes"]["places"]))
        except KeyError: 
            continue

tweet_frame = pd.concat(tweet_frames).rename(columns = {
    "id": "tweet_id", 
    "public_metrics": "tweet_metrics"}).reset_index(drop=True)

user_frame = pd.concat(user_frames).rename(columns = {
    "id": "author_id", 
    "public_metrics": "author_metrics"}).drop("created_at", axis = 1).reset_index(drop=True)

geo_frame = pd.concat(geo_frames).rename(columns = {"id": "place_id"})

tweet_frame = (tweet_frame.merge(pd.json_normalize(tweet_frame.tweet_metrics), left_index= True, right_index = True)
    .merge(pd.json_normalize(tweet_frame.geo), left_index= True, right_index = True)
    .drop(["tweet_metrics", "geo"], axis = 1)
    .rename(columns = {"coordinates.type": "coordinates_type", "coordinates.coordinates": "coordinates"}))

user_frame = user_frame.merge(pd.json_normalize(user_frame.author_metrics), 
    left_index= True, right_index = True).drop("author_metrics", axis = 1)

tweet_frame = pd.merge(
    tweet_frame, 
    user_frame, 
    left_on = "author_id", 
    right_on = "author_id", 
    how="left").drop_duplicates("tweet_id").reset_index(drop=True)

tweet_frame = pd.merge(
    tweet_frame,
    geo_frame,
    left_on = "place_id",
    right_on = "place_id", 
    how="left").drop_duplicates("tweet_id").reset_index(drop=True)

tweet_frame.tweet_id = tweet_frame.tweet_id.astype("int64")
tweet_frame = pd.merge(
    tweet_frame,
    naacl_srw_2016,
    left_on = "tweet_id",
    right_on = "tweet_id").drop_duplicates("tweet_id").reset_index(drop=True)

tweet_frame.to_csv("../../data/Waseem/tweets_with_labels.csv", index = False)

In [48]:
tweet_frame = pd.DataFrame(data["data"]).rename(columns = {
    "id": "tweet_id", 
    "public_metrics": "tweet_metrics"})

user_frame = pd.DataFrame(data["includes"]["users"]).rename(columns = {
    "id": "author_id", 
    "public_metrics": "author_metrics"}).drop("created_at", axis = 1)

geo_frame = pd.DataFrame(data["includes"]["places"]).rename(columns = {"id": "place_id"})

tweet_frame.merge(user_frame, on = "author_id")

,text,created_at,reply_settings,tweet_metrics,conversation_id,lang,author_id,tweet_id,source,geo,name,author_metrics,username,description,verified
0,So Drasko just said he was impressed the girls...,2015-03-02T10:29:34.000Z,everyone,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",572342978255048705,en,2498963143,572342978255048705,Twitter Web App,NaN,thefoxbandit,"{'followers_count': 7, 'following_count': 0, '...",thefoxbandit,"pro gamer,drawer, full time nerd and emo",False
1,Drasko they didn't cook half a bird you idiot ...,2015-03-02T10:23:41.000Z,everyone,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",572341498827522049,en,110114783,572341498827522049,Twitter for iPhone,"{'place_id': '017453ae077eafd3', 'coordinates'...",patricia hilder,"{'followers_count': 13, 'following_count': 62,...",trish2295,,False
2,Hopefully someone cooks Drasko in the next ep ...,2015-03-02T10:19:37.000Z,everyone,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",572340476503724032,en,38650214,572340476503724032,Twitter for iPhone,NaN,"food bling, Brisbane","{'followers_count': 3324, 'following_count': 2...",foodbling,"food, wine, gin, good tunes, travel, football ...",False
3,of course you were born in serbia...you're as ...,2015-03-02T09:56:43.000Z,everyone,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",572334712804384768,en,2587278392,572334712804384768,Twitter Web Client,NaN,Bossbeasty91,"{'followers_count': 491, 'following_count': 22...",lilbeastunleash,Psn -Bossbeasty91 ...,False
4,These girls are the equivalent of the irritati...,2015-03-02T09:48:33.000Z,everyone,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",572332655397629952,en,2601524623,572332655397629952,Twitter Web Client,NaN,CreatetheWorldYouWant 💉💉💉💉,"{'followers_count': 2246, 'following_count': 2...",JL_Whitaker,"I write books https://t.co/hpyRcVzqy1 ,Cancer ...",False


In [35]:
pd.DataFrame(data["includes"]["users"])

,name,id,public_metrics,created_at,username,description,verified
0,thefoxbandit,2498963143,"{'followers_count': 7, 'following_count': 0, '...",2014-05-16T13:51:44.000Z,thefoxbandit,"pro gamer,drawer, full time nerd and emo",False
1,patricia hilder,110114783,"{'followers_count': 13, 'following_count': 62,...",2010-01-31T11:35:37.000Z,trish2295,,False
2,"food bling, Brisbane",38650214,"{'followers_count': 3324, 'following_count': 2...",2009-05-08T11:45:45.000Z,foodbling,"food, wine, gin, good tunes, travel, football ...",False
3,Bossbeasty91,2587278392,"{'followers_count': 491, 'following_count': 22...",2014-06-25T10:30:25.000Z,lilbeastunleash,Psn -Bossbeasty91 ...,False
4,CreatetheWorldYouWant 💉💉💉💉,2601524623,"{'followers_count': 2246, 'following_count': 2...",2014-07-03T11:27:16.000Z,JL_Whitaker,"I write books https://t.co/hpyRcVzqy1 ,Cancer ...",False
